# Police Shootings in the US: An Analysis

## Introduzione
In questo progetto analizzeremo i dati relativi alle sparatorie della polizia negli Stati Uniti. L'obiettivo principale è quello di identificare potenziali bias razziali e altre correlazioni significative. Utilizzeremo diversi dataset che contengono informazioni sui casi di sparatorie fatali, le condizioni socio-economiche delle città, e la composizione etnica delle popolazioni. In particolare, cercheremo di rispondere alle seguenti domande:

1. Esistono differenze significative nella frequenza delle sparatorie della polizia tra diverse etnie?
2. Ci sono differenze regionali nei tassi di sparatorie della polizia?

## Descrizione dei Dataset
I dataset utilizzati in questa analisi sono:
- **Police Killings US**: informazioni aggiuntive sulle sparatorie della polizia.
- **Share Race By City**: composizione etnica delle città.
- **Population By City**: popolazione delle città.

Analizzeremo le variabili più importanti di ciascun dataset e discuteremo le loro caratteristiche principali.

In [61]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set up visualization style
sns.set(style="whitegrid")

In [62]:
# Define file paths
file_paths = {
    'killings': 'C:/Work/Lavoro/Start2impact/Data Visualization and Manipulation/archive/PoliceKillingsUS.csv',
    'race_share': 'C:/Work/Lavoro/Start2impact/Data Visualization and Manipulation/archive/ShareRaceByCity.csv',
    'population': 'C:/Work/Lavoro/Start2impact/Data Visualization and Manipulation/archive/PopulationByCity.csv',
}

# Function to load CSV files handling UnicodeDecodeError
def load_csv(file_path):
    try:
        return pd.read_csv(file_path)
    except UnicodeDecodeError:
        return pd.read_csv(file_path, encoding='latin1')

# Create variables for each dataset
killings = load_csv(file_paths['killings'])
race_share = load_csv(file_paths['race_share'])
population = load_csv(file_paths['population'])

print("\nDataset: killings")
print(killings.head())
print(killings.columns)

print("\nDataset: race_share")
print(race_share.head())
print(race_share.columns)

print("\nDataset: population")
print(population.head())
print(population.columns)


Dataset: killings
   id                name      date   manner_of_death       armed   age  \
0   3          Tim Elliot  02/01/15              shot         gun  53.0   
1   4    Lewis Lee Lembke  02/01/15              shot         gun  47.0   
2   5  John Paul Quintero  03/01/15  shot and Tasered     unarmed  23.0   
3   8     Matthew Hoffman  04/01/15              shot  toy weapon  32.0   
4   9   Michael Rodriguez  04/01/15              shot    nail gun  39.0   

  gender race           city state  signs_of_mental_illness threat_level  \
0      M    A        Shelton    WA                     True       attack   
1      M    W          Aloha    OR                    False       attack   
2      M    H        Wichita    KS                    False        other   
3      M    W  San Francisco    CA                     True       attack   
4      M    H          Evans    CO                    False       attack   

          flee  body_camera  
0  Not fleeing        False  
1  Not fleein

In [63]:
#verifica dei valori mancanti in ogni dataset
# 6. killings
print("\nValori mancanti in killings:")
print(killings.isnull().sum())

# 7. race_share
print("\nValori mancanti in race_share:")
print(race_share.isnull().sum())

# 8. population
print("\nValori mancanti in population:")
print(population.isnull().sum())


Valori mancanti in killings:
id                           0
name                         0
date                         0
manner_of_death              0
armed                        9
age                         77
gender                       0
race                       195
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                        65
body_camera                  0
dtype: int64

Valori mancanti in race_share:
Geographic area          0
City                     0
share_white              0
share_black              0
share_native_american    0
share_asian              0
share_hispanic           0
dtype: int64

Valori mancanti in population:
city            0
state           0
population      0
lat           604
lng           604
dtype: int64


In [64]:
print(killings['race'].unique())

# Sostituire i valori NaN nella colonna 'race' con 'Unknown'
killings['race'].fillna('Unknown', inplace=True)

# Verificare che la sostituzione sia avvenuta correttamente
print("Numero di valori mancanti nella colonna 'race' dopo la sostituzione:")
print(killings['race'].isna().sum())

# Visualizzare i valori unici della colonna 'race' per confermare
print("Valori unici nella colonna 'race' dopo la sostituzione:")
print(killings['race'].unique())

# Visualizzare i primi 5 valori per confermare
print(killings.head())

['A' 'W' 'H' 'B' 'O' nan 'N']
Numero di valori mancanti nella colonna 'race' dopo la sostituzione:
0
Valori unici nella colonna 'race' dopo la sostituzione:
['A' 'W' 'H' 'B' 'O' 'Unknown' 'N']
   id                name      date   manner_of_death       armed   age  \
0   3          Tim Elliot  02/01/15              shot         gun  53.0   
1   4    Lewis Lee Lembke  02/01/15              shot         gun  47.0   
2   5  John Paul Quintero  03/01/15  shot and Tasered     unarmed  23.0   
3   8     Matthew Hoffman  04/01/15              shot  toy weapon  32.0   
4   9   Michael Rodriguez  04/01/15              shot    nail gun  39.0   

  gender race           city state  signs_of_mental_illness threat_level  \
0      M    A        Shelton    WA                     True       attack   
1      M    W          Aloha    OR                    False       attack   
2      M    H        Wichita    KS                    False        other   
3      M    W  San Francisco    CA                  

In [65]:
# Identificare città con due o più nomi
race_share['city_parts'] = race_share['City'].apply(lambda x: len(x.split()))
multi_name_cities = race_share[race_share['city_parts'] > 1]['City']
print("Città composte da due o più nomi:")
print(multi_name_cities)

# Estrazione dell'ultima parola da ogni nome nella colonna "City"
race_share['last_word'] = race_share['City'].apply(lambda x: x.split()[-1])

# Conteggio delle frequenze di ciascuna ultima parola
suffix_counts = race_share['last_word'].value_counts()

# Visualizzare i suffissi più comuni
print(suffix_counts)

Città composte da due o più nomi:
0                    Abanda CDP
1                Abbeville city
2               Adamsville city
3                  Addison town
4                    Akron town
                  ...          
29263    Woods Landing-Jelm CDP
29264              Worland city
29265               Wright town
29266                Yoder town
29267             Y-O Ranch CDP
Name: City, Length: 29267, dtype: object
last_word
city            10154
CDP              9620
town             4361
village          3774
borough          1209
County)           120
government          8
(balance)           7
municipality        4
Counties)           3
County              2
City                2
Washington          1
county              1
Bow                 1
corporation         1
Name: count, dtype: int64


In [66]:
# Rinominare le colonne in race_share per uniformità
race_share.rename(columns={'City': 'city','Geographic area': 'state'}, inplace=True)

# Rimuovere i suffissi dai nomi delle città in race_share
race_share['city'] = race_share['city'].str.split(',').str[0]

print(race_share)

      state                    city share_white share_black  \
0        AL              Abanda CDP        67.2        30.2   
1        AL          Abbeville city        54.4        41.4   
2        AL         Adamsville city        52.3        44.9   
3        AL            Addison town        99.1         0.1   
4        AL              Akron town        13.2        86.5   
...     ...                     ...         ...         ...   
29263    WY  Woods Landing-Jelm CDP        95.9           0   
29264    WY            Worland city        89.9         0.3   
29265    WY             Wright town        94.5         0.1   
29266    WY              Yoder town        97.4           0   
29267    WY           Y-O Ranch CDP        92.8         1.5   

      share_native_american share_asian share_hispanic  city_parts last_word  
0                         0           0            1.6           2       CDP  
1                       0.1           1            3.1           2      city  
2     

In [67]:
# Dividere i nomi delle città e tenere tutto tranne l'ultima parola
race_share['city'] = race_share['city'].str.rsplit(' ', n=1).str[0]

In [68]:
print(race_share)
print(population)
print(killings)

      state                city share_white share_black share_native_american  \
0        AL              Abanda        67.2        30.2                     0   
1        AL           Abbeville        54.4        41.4                   0.1   
2        AL          Adamsville        52.3        44.9                   0.5   
3        AL             Addison        99.1         0.1                     0   
4        AL               Akron        13.2        86.5                     0   
...     ...                 ...         ...         ...                   ...   
29263    WY  Woods Landing-Jelm        95.9           0                     0   
29264    WY             Worland        89.9         0.3                   1.3   
29265    WY              Wright        94.5         0.1                   1.4   
29266    WY               Yoder        97.4           0                     0   
29267    WY           Y-O Ranch        92.8         1.5                   2.6   

      share_asian share_his

In [69]:
# Controllare quante righe hanno valore NaN nella colonna 'city'
nan_rows = race_share['city'].isna().sum()
print(f"Numero di righe con valore NaN nella colonna 'city': {nan_rows}")
nan_rows = population['city'].isna().sum()
print(f"Numero di righe con valore NaN nella colonna 'city': {nan_rows}")
nan_rows = killings['city'].isna().sum()
print(f"Numero di righe con valore NaN nella colonna 'city': {nan_rows}")

Numero di righe con valore NaN nella colonna 'city': 0
Numero di righe con valore NaN nella colonna 'city': 0
Numero di righe con valore NaN nella colonna 'city': 0


In [70]:
# Mappare le etnie con le loro abbreviazioni
ethnicity_map = {
    'W': 'white',
    'B': 'black',
    'N': 'native_american',
    'A': 'asian',
    'H': 'hispanic',
    'O': 'other',
    'Unknown': 'unknown'
}

# Aggiungere una colonna di popolazione etnica aggregata
def map_ethnicity(race):
    races = race.split(';')
    return [ethnicity_map[r] for r in races if r in ethnicity_map]

# Aggiungere le colonne con le etnie mappate
killings['mapped_race'] = killings['race'].apply(lambda x: map_ethnicity(str(x)))

print(killings)

        id                name      date   manner_of_death       armed   age  \
0        3          Tim Elliot  02/01/15              shot         gun  53.0   
1        4    Lewis Lee Lembke  02/01/15              shot         gun  47.0   
2        5  John Paul Quintero  03/01/15  shot and Tasered     unarmed  23.0   
3        8     Matthew Hoffman  04/01/15              shot  toy weapon  32.0   
4        9   Michael Rodriguez  04/01/15              shot    nail gun  39.0   
...    ...                 ...       ...               ...         ...   ...   
2530  2822    Rodney E. Jacobs  28/07/17              shot         gun  31.0   
2531  2813               TK TK  28/07/17              shot     vehicle   NaN   
2532  2818  Dennis W. Robinson  29/07/17              shot         gun  48.0   
2533  2817       Isaiah Tucker  31/07/17              shot     vehicle  28.0   
2534  2815        Dwayne Jeune  31/07/17              shot       knife  32.0   

     gender     race           city sta

In [71]:
# Calcolare il numero di decessi per ogni etnia in ogni città
death_counts = killings.explode('mapped_race').groupby(['city', 'mapped_race']).size().unstack(fill_value=0)

# Visualizzare i primi risultati
print(death_counts.head())
# Verifica delle dimensioni del dataset unito
print(f"Dimensioni del dataset unito: {death_counts.shape}")

mapped_race  asian  black  hispanic  native_american  other  unknown  white
city                                                                       
Abbeville        0      1         0                0      0        0      0
Abilene          0      0         0                0      0        0      1
Abingdon         0      0         0                0      0        0      1
Acworth          0      0         0                0      0        0      1
Addison          0      0         1                0      0        0      0
Dimensioni del dataset unito: (1417, 7)


In [72]:
# Unione dei dataset race_share e population basata su city e state
race_population = pd.merge(race_share, population, on=['state', 'city'], how='left')

# Verifica delle dimensioni del dataset
print(f"Dimensioni del dataset 'race_population': {race_population.shape}")
print(race_population.head())

# Converti le colonne in tipo numerico se necessario
race_population['population'] = pd.to_numeric(race_population['population'], errors='coerce')
ethnicities = ['white', 'black', 'native_american', 'asian', 'hispanic', 'other', 'unknown']
for race in ethnicities:
    col_name = f'share_{race}'
    if col_name not in race_population.columns:
        race_population[col_name] = 0
    race_population[col_name] = pd.to_numeric(race_population[col_name], errors='coerce')

# Calcola la popolazione per etnia
for race in ethnicities:
    race_population[f'{race}_population'] = (race_population[f'share_{race}'] / 100) * race_population['population']

# Elimina le righe dove i valori delle popolazioni etniche o della popolazione totale sono NaN
columns_to_check = ['population'] + [f'{race}_population' for race in ethnicities]
race_population_clean = race_population.dropna(subset=columns_to_check)

# Verifica delle dimensioni del dataset con le popolazioni etniche calcolate
print(f"Dimensioni del dataset 'race_population_clean' con popolazioni etniche calcolate: {race_population_clean.shape}")
print(race_population_clean.head())

Dimensioni del dataset 'race_population': (29268, 12)
  state        city share_white share_black share_native_american share_asian  \
0    AL      Abanda        67.2        30.2                     0           0   
1    AL   Abbeville        54.4        41.4                   0.1           1   
2    AL  Adamsville        52.3        44.9                   0.5         0.3   
3    AL     Addison        99.1         0.1                     0         0.1   
4    AL       Akron        13.2        86.5                     0           0   

  share_hispanic  city_parts last_word  population      lat      lng  
0            1.6           2       CDP         NaN      NaN      NaN  
1            3.1           2      city      2567.0  31.5664 -85.2528  
2            2.3           2      city         NaN      NaN      NaN  
3            0.4           2      town         NaN      NaN      NaN  
4            0.3           2      town         NaN      NaN      NaN  
Dimensioni del dataset 'race_popu

In [73]:
# Unisci i dati delle popolazioni etniche con i dati di conteggio dei decessi
death_rates = pd.merge(race_population_clean, death_counts, left_on='city', right_index=True, how='left').fillna(0)

# Verifica delle dimensioni del dataset unito
print(f"Dimensioni del dataset unito: {death_rates.shape}")

# Visualizza le colonne con le popolazioni etniche calcolate
ethnic_pop_columns = [f'{race}_population' for race in ethnicities]
print(death_rates[['city'] + ethnic_pop_columns].head())

Dimensioni del dataset unito: (2245, 28)
           city  white_population  black_population  \
1     Abbeville          1396.448          1062.738   
6   Albertville         28249.980           707.180   
14     Anniston         33600.543         38712.035   
15         Arab          7963.704             8.244   
18       Ariton           578.637           141.888   

    native_american_population  asian_population  hispanic_population  \
1                        2.567            25.670               79.577   
6                      297.760           186.100            10384.380   
14                     225.507           601.352             2029.563   
15                      49.464            57.708              140.148   
18                       0.000             0.739               44.340   

    other_population  unknown_population  
1                0.0                 0.0  
6                0.0                 0.0  
14               0.0                 0.0  
15               

In [74]:
# Step 1: Rimuoviamo le colonne non necessarie
columns_to_drop = ['share_white', 'share_black', 'share_native_american', 'share_asian', 'share_hispanic', 'share_other', 'share_unknown', 'city_parts', 'last_word', 'lat', 'lng']
death_rates_clean = death_rates.drop(columns=columns_to_drop, errors='ignore')

# Step 2: Raggruppiamo i dati per stato e sommiamo le popolazioni etniche
state_population = death_rates_clean.groupby('state').sum(numeric_only=True).reset_index()

# Step 3: Visualizziamo il risultato
print(f"Dimensioni del dataset 'state_population': {state_population.shape}")
print(state_population.head())


Dimensioni del dataset 'state_population': (51, 16)
  state  population  white_population  black_population  \
0    AK    353942.0      2.358912e+05         20555.124   
1    AL   2695464.0      1.291734e+06       1251646.585   
2    AR   1570573.0      1.031075e+06        366894.512   
3    AZ   8225324.0      5.769607e+06        447390.437   
4    CA  49091540.0      2.617798e+07       3700357.433   

   native_american_population  asian_population  hispanic_population  \
0                   32071.437         23734.823         2.646928e+04   
1                    8836.199         45145.889         1.091254e+05   
2                   12246.550         41227.878         1.369624e+05   
3                  197831.342        265737.927         2.883037e+06   
4                  420370.620       7163791.171         1.944776e+07   

   other_population  unknown_population  asian  black  hispanic  \
0               0.0                 0.0    1.0   16.0       6.0   
1               0.0       

In [75]:
# Calcola il tasso di mortalità per ciascuna etnia
for race in ethnicities:
    state_population[f'{race}_death_rate'] = (state_population[race] / state_population[f'{race}_population']) * 100000  # per 100,000 abitanti

# Sostituisci i valori NaN con 0
state_population = state_population.fillna(0)

# Sostituisci i valori Inf e -Inf con 0
state_population = state_population.replace([np.inf, -np.inf], 0)

# Visualizza i primi risultati per verificare le sostituzioni
print(state_population.head())


  state  population  white_population  black_population  \
0    AK    353942.0      2.358912e+05         20555.124   
1    AL   2695464.0      1.291734e+06       1251646.585   
2    AR   1570573.0      1.031075e+06        366894.512   
3    AZ   8225324.0      5.769607e+06        447390.437   
4    CA  49091540.0      2.617798e+07       3700357.433   

   native_american_population  asian_population  hispanic_population  \
0                   32071.437         23734.823         2.646928e+04   
1                    8836.199         45145.889         1.091254e+05   
2                   12246.550         41227.878         1.369624e+05   
3                  197831.342        265737.927         2.883037e+06   
4                  420370.620       7163791.171         1.944776e+07   

   other_population  unknown_population  asian  ...  other  unknown  white  \
0               0.0                 0.0    1.0  ...    0.0      2.0    9.0   
1               0.0                 0.0    0.0  ...    0

In [76]:
# Calcola il totale dei morti per ogni stato
state_population['total_deaths'] = state_population[['white', 'black', 'native_american', 'asian', 'hispanic', 'other', 'unknown']].sum(axis=1)

# Visualizza il risultato
print(state_population[['state', 'total_deaths']])

   state  total_deaths
0     AK          39.0
1     AL          70.0
2     AR          63.0
3     AZ         129.0
4     CA         446.0
5     CO          86.0
6     CT           6.0
7     DC          11.0
8     DE          14.0
9     FL         155.0
10    GA         111.0
11    HI           7.0
12    IA          18.0
13    ID          10.0
14    IL          91.0
15    IN          58.0
16    KS          36.0
17    KY          46.0
18    LA          60.0
19    MA          24.0
20    MD          50.0
21    ME          19.0
22    MI          46.0
23    MN          59.0
24    MO          83.0
25    MS          55.0
26    MT          12.0
27    NC         118.0
28    ND           3.0
29    NE          24.0
30    NH           7.0
31    NJ          29.0
32    NM          51.0
33    NV          40.0
34    NY          35.0
35    OH         100.0
36    OK          90.0
37    OR          59.0
38    PA          58.0
39    RI           2.0
40    SC          66.0
41    SD          10.0
42    TN   

In [77]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import json
import os
from folium.features import Choropleth
from IPython.display import display, HTML
from io import BytesIO
import base64

# Supponiamo che il dataset sia già caricato in state_population
# Assicurati che non ci siano NaN o Inf
state_population = state_population.fillna(0)
state_population = state_population.replace([np.inf, -np.inf], 0)

# Percorso del file GeoJSON
geojson_path = r'C:\Work\Lavoro\Start2impact\Data Visualization and Manipulation\archive_GeoJSON\us-states.json'

# Carica i dati geografici degli Stati Uniti
try:
    with open(geojson_path) as f:
        us_states_geo = json.load(f)
    print("File GeoJSON caricato correttamente")
except Exception as e:
    print(f"Errore nel caricamento del file GeoJSON: {e}")

# Mappa dei nomi degli stati alle loro abbreviazioni
us_states_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT',
    'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI',
    'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Inverti il dizionario per mappare le abbreviazioni ai nomi completi
abbrev_us_states = {v: k for k, v in us_states_abbrev.items()}

# Usa il metodo map per sostituire le abbreviazioni con i nomi completi
state_population['state'] = state_population['state'].map(abbrev_us_states)

# Verifica che tutte le abbreviazioni siano state mappate correttamente
if state_population['state'].isnull().any():
    print("Alcuni stati non sono stati mappati correttamente. Controlla i dati:")
    print(state_population[state_population['state'].isnull()])

# Funzione per creare una heatmap interattiva per una specifica etnia
def create_interactive_map(state_population, column, title):
    # Crea una mappa centrata sugli Stati Uniti
    m = folium.Map(location=[37.8, -96], zoom_start=4)

    # Aggiungi il layer dei tassi di mortalità
    Choropleth(
        geo_data=us_states_geo,
        name='choropleth',
        data=state_population,
        columns=['state', column],  # Usa 'state' invece di 'state_abbrev'
        key_on='feature.properties.name',  # Usa 'feature.properties.name' invece di 'feature.id'
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'{title} per 100,000 abitanti'
    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m

# Lista delle etnie
ethnicities = ['white', 'black', 'native_american', 'asian', 'hispanic', 'other', 'unknown']

# Crea una mappa interattiva per ogni etnia e visualizzala
for race in ethnicities:
    m = create_interactive_map(state_population, f'{race}_death_rate', f'Tasso di mortalità {race.capitalize()}')
    display(m)

# Crea una mappa interattiva del numero totale delle morti per stato e visualizzala
m = create_interactive_map(state_population, 'total_deaths', 'Numero totale delle morti per stato')
display(m)


File GeoJSON caricato correttamente


In [78]:
# Funzione per creare un'immagine delle barre dei tassi di mortalità per le etnie
def create_bar_image(data):
    fig, ax = plt.subplots()
    ax.bar(data.index, data.values, color=['blue', 'green', 'red', 'purple', 'orange', 'brown'])
    plt.xticks(rotation=90)
    plt.tight_layout()
    
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    image = base64.b64encode(buf.read()).decode('utf8')
    return image

# Funzione per creare una mappa con grafici a barre per rappresentare la mortalità per etnia
def create_bar_map(state_population, geojson_data, title):
    # Crea una mappa centrata sugli Stati Uniti
    m = folium.Map(location=[37.8, -96], zoom_start=4)

    # Estrai le coordinate dei centri degli stati dal file GeoJSON
    state_coords = {}
    for feature in geojson_data['features']:
        state_name = feature['properties']['name']
        geometry = feature['geometry']
        if geometry['type'] == 'Polygon':
            lon, lat = np.mean(np.array(geometry['coordinates'][0]), axis=0)
        elif geometry['type'] == 'MultiPolygon':
            lon, lat = np.mean(np.concatenate(geometry['coordinates'][0]), axis=0)
        state_coords[state_name] = [lat, lon]

    # Aggiungi i marker con grafici a barre
    for idx, row in state_population.iterrows():
        state = row['state']
        coords = state_coords.get(state)
        if coords:
            ethnicity_data = row[['white_death_rate', 'black_death_rate', 'asian_death_rate', 'native_american_death_rate', 'hispanic_death_rate', 'other_death_rate']]
            img_data = create_bar_image(ethnicity_data)
            html = f'<img src="data:image/png;base64,{img_data}">'
            iframe = folium.IFrame(html, width=300, height=200)
            popup = folium.Popup(iframe, max_width=300)
            folium.Marker(location=coords, popup=popup).add_to(m)
    
    folium.LayerControl().add_to(m)
    
    return m

# Crea una mappa con grafici a barre per etnia
bar_map = create_bar_map(state_population, us_states_geo, 'Tassi di mortalità per etnia per stato')

# Mostra la mappa nel notebook
display(bar_map)



## Conclusioni:

Gli stati americani più colpiti da omicidi da parte delle forze dell'ordine sono California, Texas e Florida. Questi stati sono tra i più diversi etnicamente di tutti gli Stati Uniti. In tutti e tre gli stati l'etnia maggiormente colpita dagli omicidi è quella afroamericana, seguita da quella ispanica per la California e la Florida, e da quella nativa americana per il Texas.